<a href="https://colab.research.google.com/github/su-sumico/seminar/blob/main/04_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="https://raw.githubusercontent.com/holoviz/holoviz/master/doc/_static/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 4. Making dashboards</h2></div>


This tutorial demonstrates the Easiest Way to Create an Interactive Dashboard in Python from any DataFrame.  If you already know some Pandas, you can almost immediately use `hvPlot` `.interactive` and `Panel` to turn your DataFrame processing pipeline into a dashboard! It just takes a few lines of familiar code to make an interactive dashboard like this:
![Data App](https://github.com/holoviz-community/HoloViz_KDD2022/blob/main/assets/easy-dataframe-dashboards.gif?raw=1)



## Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot`.

In [1]:
def environment():
    try:
        get_ipython()
        print('notebook')
        return str(get_ipython())
    except:
        print('server')
        return 'server'
env = environment()

if env=="server":
   theme="fast"
else:
   theme="simple"

if 'google.colab' in env:
    print('Running on CoLab')
    !pip install panel hvplot

notebook
Running on CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.1 MB/s eta 0:00:00


In [2]:
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

In [3]:
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

## Load Data

In [4]:
from bokeh.sampledata.autompg import autompg_clean as df
df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,North America,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,North America,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,North America,ford torino,ford


## Define DataFrame Pipeline

In [5]:
(
    df[
        (df.cyl == 4) &
        (df.mfr.isin(['ford','chevrolet']))
    ]
    .groupby(['origin', 'cyl', 'mfr', 'yr'])['hp'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='yr')
).head(1)

,origin,cyl,mfr,yr,hp
0,North America,4,chevrolet,71,81.0


## Make DataFrame Pipeline Interactive

What if we would like to turn the values of cyl, the values of mfr, and the variable hp into interactive widgets that we can change and control? Is it possible? Yes, it is, and with hvPlot it’s not even difficult. Here are the steps:

- First, we need to wrap our dataframe with `.interactive()`: `idf = df.interactive()`, so that this dataframe becomes interactive and we can use Panel widgets on this dataframe.

“.interactive stores a copy of your pipeline (series of method calls or other expressions on your data) and dynamically replays the pipeline whenever that widget changes.”


In [6]:
idf = df.interactive()

Define [Panel widgets](https://panel.holoviz.org/reference/index.html#widgets)

- Second, we can define the panel widgets we would like to use. Here I defined a panel widget for cylinders, a widget for the manufacturer, and a widget to select the y axis.


In [7]:
cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
cylinders

IntSlider(end=8, name='Cylinders', sizing_mode='stretch_width', start=4, step=2, value=4)

In [8]:
mfr = pn.widgets.ToggleGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    button_type='success')
mfr

CheckButtonGroup(button_type='success', name='MFR', options=['ford', 'chevrolet', ...], sizing_mode='stretch_width', value=['ford', 'chevrolet', ...])

In [9]:
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['hp', 'weight'],
    button_type='success'
)
yaxis

RadioButtonGroup(button_type='success', name='Y axis', options=['hp', 'weight'], sizing_mode='stretch_width', value='hp')

- Finally, we can replace the values or variables from the original Pandas pipeline to these widgets we just defined. We define the output of the pipeline as ipipeline:


In [10]:
ipipeline = (
    idf[
        (idf.cyl == cylinders) &
        (idf.mfr.isin(mfr))
    ]
    .groupby(['origin', 'mpg'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='mpg')
    .reset_index(drop=True)
)
ipipeline.head()

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

## Pipe to Table

In [11]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme=theme)
itable

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

Check out the [Tabulator Reference Guide](https://panel.holoviz.org/reference/widgets/Tabulator.html) for more inspiration.

## Pipe to hvplot

In [12]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
)

<ipython-input-12-0bed645b6b2f>:3: PanelDeprecationWarning: "HSpacer(..., background='#ff6f69')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ff6f69'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[0]),
<ipython-input-12-0bed645b6b2f>:4: PanelDeprecationWarning: "HSpacer(..., background='#ffcc5c')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ffcc5c'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[1]),
<ipython-input-12-0bed645b6b2f>:5: PanelDeprecationWarning: "HSpacer(..., background='#88d8b0')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#88d8b0'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[2]),


Row(sizing_mode='stretch_width')
    [0] HSpacer(background='#ff6f69', height=50, styles={'background': '#ff6f69'})
    [1] HSpacer(background='#ffcc5c', height=50, styles={'background': '#ffcc5c'})
    [2] HSpacer(background='#88d8b0', height=50, styles={'background': '#88d8b0'})

In [13]:
ihvplot = ipipeline.hvplot(x='mpg', y=yaxis, by='origin', color=PALETTE, line_width=6, height=400)
ihvplot

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

/usr/local/lib/python3.10/dist-packages/holoviews/core/spaces.py:497: ParamFutureWarning: Parameter 'callable' on <class 'holoviews.core.spaces.Callable'> is being given a valid parameter reference <function Interactive._callback.<locals>.evaluate at 0x7f442e068af0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function Interactive._callback.<locals>.evaluate at 0x7f442e068af0> will be resolved to its underlying value :NdOverlay   [origin]
   :Curve   [mpg]   (hp). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(callable=callable,


## Layout using Panel

In [14]:
pn.Column(
    pn.Row(
        cylinders, mfr, yaxis
    ),
    ihvplot.panel(),
    itable.panel()
)

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] IntSlider(end=8, name='Cylinders', sizing_mode='stretch_width', start=4, step=2, value=4)
        [1] CheckButtonGroup(button_type='success', name='MFR', options=['ford', 'chevrolet', ...], sizing_mode='stretch_width', value=['ford', 'chevrolet', ...])
        [2] RadioButtonGroup(button_type='success', name='Y axis', options=['hp', 'weight'], sizing_mode='stretch_width', value='hp')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Tabulator, defer_load=False, sizing_mode='stretch_width')

## Layout using Template

Here we use the [FastListTemplate](https://panel.holoviz.org/reference/templates/FastListTemplate.html#templates-gallery-fastlisttemplate).

In [15]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive',
    sidebar=[cylinders, 'Manufacturers', mfr, 'Y axis' , yaxis],
    main=[ihvplot.panel(), itable.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

Please note that to get the Tabulator table styled nicely for dark mode you can set `theme='fast'` when you define the `itable`. It won't work in the notebook though.

To *serve the notebook* run `panel serve 04_Dashboard.ipynb`.

<div class="alert alert-info">
<h4>Exercise</h4>
    
Take a look at the [Panel widget docs](https://panel.holoviz.org/reference/index.html#widgets). And change the three widgets above to different types of widgets of your choice.  
        
<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python

cylinders = pn.widgets.IntInput(name='Cylinders', value=4, step=2, start=4, end=8)
mfr = pn.widgets.CheckBoxGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi']
)
yaxis = pn.widgets.RadioBoxGroup(
    name='Y axis',
    options=['hp', 'weight'],
)
    
    
```

</details>
</div>

<div class="alert alert-info">
<h4>Exercise</h4>
    
Add a widget to select origin in the example above.

<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python

origin = pn.widgets.ToggleGroup(
    name='Origin',
    options=['North America', 'Asia', 'Europe'],
    value=['North America', 'Asia', 'Europe'],
    button_type='success')
origin
    
ipipeline = (
    idf[
        (idf.cyl == cylinders) &
        (idf.mfr.isin(mfr)) &
        (idf.origin.isin(origin))
    ]
    .groupby(['origin', 'mpg'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='mpg')  
    .reset_index(drop=True)
)
ipipeline.head()
    
```

</details>
</div>

<div class="alert alert-info">
<h4>Exercise</h4>
    
Try save this notebook to a Python file, or copy and paste the relevant code to a Python file.
    
Then serve the dashboard from the Python file: `panel serve 04_Dashboard.py`.

</details>
</div>

<div class="alert alert-info">
<h4>Reading time </h4>
    
Read the [Panel documentation](https://panel.holoviz.org/) and let us know if you have any questions.
</div>